In [2]:
import pandas as pd
from urllib.parse import urlparse
import re

Cleaning Work-Ziwei Xu-500954406

In [3]:
def extract_title_from_url_expert(url):
    path = urlparse(url).path
    title = path.split('/')[-1]
    return title.replace('-', ' ').title()
df_expert = pd.read_csv('ExpertReviewsClean43LIWC.txt', delimiter='\t', encoding='ISO-8859-1')
df_meta = pd.read_csv('metaClean43Brightspace.txt', delimiter='\t', encoding='ISO-8859-1')
df_MovieID = pd.read_csv('allMoviesWithUniqueID.csv', sep=None, engine='python', encoding='ISO-8859-1')

# I named the dataframe of Jayy's table as "df_MovieTable" and I change the column name to "Movie_ID" for it was "ID" in the csv
df_MovieID = df_MovieID.rename(columns={"ID": "Movie_ID"})
df_expert['title'] = df_expert['url'].apply(extract_title_from_url_expert).str.lower()


In [4]:
df_expert['ExpertReview_ID'] = range(1, len(df_expert) + 1)

In [5]:
def clean_text_Meta_Expert(text):
    if pd.isna(text): 
        return ''  
    text = text.lower()
    # Remove special characters using regex (keeping only alphanumeric characters and spaces)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^\w]', '', text)
    return text

df_expert['normalized_title'] = df_expert['title'].apply(clean_text_Meta_Expert)
df_meta['normalized_title'] = df_meta['title'].apply(clean_text_Meta_Expert)
df_MovieID['normalized_title'] = df_MovieID['TITLE'].apply(clean_text_Meta_Expert)

df_expert['normalized_title'] = df_expert['normalized_title'].fillna('')
df_meta['normalized_title'] = df_meta['normalized_title'].fillna('')
df_MovieID['normalized_title'] = df_MovieID['normalized_title'].fillna('')

df_expert = df_expert.merge(df_MovieID[['normalized_title', 'Movie_ID']], 
                                          on='normalized_title', how='left')
df_meta = df_meta.merge(df_MovieID[['normalized_title', 'Movie_ID']], 
                                          on='normalized_title', how='left')


In [6]:
unmatched_expert = df_expert[df_expert['Movie_ID'].isna()]
unmatched_meta = df_meta[df_meta['Movie_ID'].isna()]

print("Unmatched rows in df_expert:")
print(unmatched_expert[['title', 'normalized_title']])

print("Unmatched rows in df_meta:")
print(unmatched_meta[['title', 'normalized_title']])

Unmatched rows in df_expert:
Empty DataFrame
Columns: [title, normalized_title]
Index: []
Unmatched rows in df_meta:
Empty DataFrame
Columns: [title, normalized_title]
Index: []


In [7]:
df_Movie_Expert = df_expert[['Movie_ID', 'ExpertReview_ID']]
df_Expert_Review = df_expert[['ExpertReview_ID', 'url', 'posemo', 'negemo']]
df_MetaTable = df_meta[['Movie_ID', 'metascore', 'RelDate']]

In [8]:
print(df_MetaTable.head())

   Movie_ID  metascore    RelDate
0     15211         70   2011/6/1
1     29944         70   2011/6/1
2     17228         76  2016/3/11
3     29945         54  2006/12/1
4     10790         61  2012/9/14


In [8]:
df_Movie_Expert.to_csv('Movie_Expert.csv', index=False, sep='\t', encoding='ISO-8859-1')
df_Expert_Review.to_csv('expert_reviews.csv', index=False, sep='\t', encoding='ISO-8859-1')
df_MetaTable.to_csv('MetaTable.csv', index=False, sep='\t', encoding='ISO-8859-1')

In [9]:
df_NewMovieTable = pd.read_csv('NewMovieTable.csv', sep=None, engine='python', encoding='ISO-8859-1')

In [11]:
import psycopg2
from psycopg2 import sql

In [12]:
conn = psycopg2.connect(
    dbname='DMGroupAssignment',
    user='postgres',
    password='201026',
    host='localhost',
    port='5432'
)

cursor = conn.cursor()

create_table_query = """
CREATE TABLE expert_reviews (
    ExpertReview_ID INTEGER PRIMARY KEY,
    Posemo FLOAT NOT NULL,
    Negemo FLOAT NOT NULL
);
"""

cursor.execute(create_table_query)

conn.commit()

insert_query = """
INSERT INTO expert_reviews (ExpertReview_ID, posemo, negemo)
VALUES ( %s, %s, %s)
ON CONFLICT (ExpertReview_ID) DO UPDATE
SET Posemo = EXCLUDED.Posemo,
    Negemo = EXCLUDED.Negemo;
"""
for index, row in df_Expert_Review.iterrows():
    cursor.execute(insert_query, (row['ExpertReview_ID'], row['posemo'], row['negemo']))


conn.commit()

cursor.close()
conn.close()

In [13]:
conn = psycopg2.connect(
    dbname='DMGroupAssignment',
    user='postgres',
    password='201026',
    host='localhost',
    port='5432'
)

cursor = conn.cursor()

create_table_query = """
CREATE TABLE Movie_Expert (
    ExpertReview_ID INTEGER PRIMARY KEY,
    Movie_ID INTEGER NOT NULL
);
"""

cursor.execute(create_table_query)

conn.commit()

insert_query = """
INSERT INTO Movie_Expert (ExpertReview_ID, Movie_ID)
VALUES ( %s, %s)
ON CONFLICT (ExpertReview_ID) DO UPDATE
SET Movie_ID = EXCLUDED.Movie_ID;
"""
for index, row in df_Movie_Expert.iterrows():
    cursor.execute(insert_query, (int(row['ExpertReview_ID']), int(row['Movie_ID'])))


conn.commit()

cursor.close()
conn.close()

In [14]:
conn = psycopg2.connect(
    dbname='DMGroupAssignment',
    user='postgres',
    password='201026',
    host='localhost',
    port='5432'
)

cursor = conn.cursor()

create_table_query = """
CREATE TABLE Meta_Table (
    Movie_ID INTEGER PRIMARY KEY,
    MetaScore INTEGER NOT NULL,
    Release_Date DATE NOT NULL
);
"""

cursor.execute(create_table_query)

conn.commit()

insert_query = """
INSERT INTO Meta_Table (Movie_ID, MetaScore, Release_Date)
VALUES ( %s, %s, %s)
ON CONFLICT (Movie_ID) DO UPDATE
SET MetaScore = EXCLUDED.MetaScore,
    Release_Date = EXCLUDED.Release_Date;
"""
for index, row in df_MetaTable.iterrows():
    cursor.execute(insert_query, (int(row['Movie_ID']), row['metascore'], row['RelDate']))


conn.commit()

cursor.close()
conn.close()

In [12]:
conn = psycopg2.connect(
    dbname='DMGroupAssignment',
    user='postgres',
    password='201026',
    host='localhost',
    port='5432'
)

cursor = conn.cursor()

create_table_query = """
CREATE TABLE New_Movie_Table (
    Movie_ID INTEGER PRIMARY KEY,
    Title TEXT NOT NULL,
    Worldwide_Box_Office FLOAT NOT NULL
);
"""

cursor.execute(create_table_query)

conn.commit()

insert_query = """
INSERT INTO New_Movie_Table (Movie_ID, Title,Worldwide_Box_Office)
VALUES ( %s, %s, %s)
ON CONFLICT (Movie_ID) DO UPDATE
SET Title = EXCLUDED.Title,
    Worldwide_Box_Office = EXCLUDED.Worldwide_Box_Office;
"""
for index, row in df_NewMovieTable.iterrows():
    cursor.execute(insert_query, (int(row['Movie_ID']), row['TITLE'],row['worldwide_box_office']))


conn.commit()

cursor.close()
conn.close()